In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import timeit
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from keras import backend as K
from keras import optimizers, backend
from keras.models import Sequential
from keras.layers import Conv1D,Conv2D, Dense, Dropout, Flatten, Activation
from keras.models import load_model
from keras.layers import RepeatVector, TimeDistributed, LSTM, GRU, Flatten, Bidirectional
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.utils.generic_utils import get_custom_objects

from keras.utils.np_utils import to_categorical
from keras.optimizers import RMSprop, adam, Nadam

Using TensorFlow backend.


In [0]:
!pip install -q xgboost

In [8]:
from google.colab import files
uploaded = files.upload()

Saving all_by_week.csv to all_by_week.csv


In [0]:
df_week = pd.read_csv("all_by_week.csv").set_index("week_firstday")
df_base = df_week/df_week[df_week !=0].min(axis=0)
#df_base = df_week/df_week[df_week !=0].quantile(0.20,axis=0)

In [10]:
df = pd.DataFrame(df_base["AO6402AL"])
df.columns = ["call_off"]
df["zero"] = ""
df.head()

,call_off,zero
week_firstday,,
2015-03-30,22.0,
2015-04-06,32.0,
2015-04-13,0.0,
2015-04-20,0.0,
2015-04-27,22.0,


In [0]:
for i in range(df.shape[0]):
    if int(df.iloc[i,0])>=1:
        df.iloc[i,1]=0
    elif int(df.iloc[i,0])==0:
        df.iloc[i,1]=1

In [61]:
df[35:47]

,call_off,zero
week_firstday,,
2015-11-30,0.0,1
2015-12-07,3.0,0
2015-12-14,5.0,0
2015-12-21,0.0,1
2015-12-28,1.0,0
2016-01-04,1.0,0
2016-01-11,4.0,0
2016-01-18,3.0,0
2016-01-25,4.0,0


In [14]:
scaler = MinMaxScaler()
train_data = scaler.fit_transform(df)
train_data[0:6]

array([[0.42307692, 0.        ],
       [0.61538462, 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.42307692, 0.        ],
       [0.19230769, 0.        ]])

In [51]:
window = 30
forecast_day = 12
training_data_rate = 0.85
feature = train_data.shape[1]

train_x = np.empty(shape=(0, window, feature))
train_y_0 = np.empty(shape=(0, forecast_day))

for i in range(len(train_data)-window-forecast_day+1):
    train_x = np.vstack((train_x, train_data[np.newaxis, i:(i+window), :]))
    train_y_0 = np.vstack((train_y_0, train_data[np.newaxis, (i+window):(i+window+forecast_day), 0]))
    # train_y_sum = (np.vstack((train_y, train_data[np.newaxis, (i+window):(i+window+forecast_day), 0]))).tolist() , will overstep train_y shape by 1, due to loop 

train_y = np.mean(train_y_0, axis=1).reshape(len(train_x),1)
    

print("train_x size: {}".format(train_x.shape))
print("train_y size: {}".format(train_y.shape))    


train_x size: (95, 30, 2)
train_y size: (95, 1)


In [0]:


y = train_y * (scaler.data_max_[0]-scaler.data_min_[0]) + scaler.data_min_[0]

In [54]:
y

array([[ 2.91666667],
       [ 2.91666667],
       [ 3.08333333],
       [ 3.08333333],
       [ 2.83333333],
       [ 2.75      ],
       [ 3.08333333],
       [ 2.91666667],
       [ 2.66666667],
       [ 3.16666667],
       [ 3.08333333],
       [ 4.41666667],
       [ 4.41666667],
       [ 6.75      ],
       [ 7.08333333],
       [ 6.33333333],
       [ 6.5       ],
       [10.66666667],
       [10.41666667],
       [10.5       ],
       [10.66666667],
       [10.58333333],
       [10.75      ],
       [ 9.5       ],
       [ 9.16666667],
       [ 7.33333333],
       [ 7.08333333],
       [ 7.83333333],
       [ 7.58333333],
       [ 4.08333333],
       [ 4.08333333],
       [ 4.16666667],
       [ 4.08333333],
       [ 3.91666667],
       [ 4.08333333],
       [ 3.91666667],
       [ 4.41666667],
       [ 3.66666667],
       [ 3.66666667],
       [ 3.        ],
       [ 3.        ],
       [ 2.41666667],
       [ 2.66666667],
       [ 3.25      ],
       [ 3.58333333],
       [ 3

In [37]:
train_y[0]

array([0.05769231, 0.03846154, 0.17307692, 0.07692308, 0.05769231,
       0.        , 0.05769231, 0.09615385, 0.        , 0.01923077,
       0.01923077, 0.07692308])

In [38]:
train_y_avg[0]

0.05608974358974359

In [67]:

train_x, validation_x = train_x[:int(len(train_x)*training_data_rate),:,:], train_x[int(len(train_x)*training_data_rate):,:,:] 
train_y, validation_y = train_y[:int(len(train_y)*training_data_rate),:], train_y[int(len(train_y)*training_data_rate):,:] 
print("X_train size: {}".format(train_x.shape))
print("y_train size: {}".format(train_y.shape))    

X_train size: (96, 12, 2)
y_train size: (96, 12, 2)


In [0]:
def swish(x):
    return ((K.sigmoid(x) * x * 1.5))
get_custom_objects().update({'swish': Activation(swish)})


In [80]:
model = Sequential()
model.add(LSTM( 256, input_shape=(window, feature), return_sequences=True))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(32, activation="relu"))
model.add(RepeatVector(12))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation="linear")))


print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 12, 256)           265216    
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_41 (Dense)             (None, 32)                4128      
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 12, 32)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 12, 32)            8320      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 12, 2)             66        
Total params: 474,850
Trainable params: 474,850
Non-trainable params: 0
_________________________________________________________________
None

In [81]:
adam = optimizers.Adam(
    lr=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=None,
    decay=0.001)


nadam = optimizers.Nadam(lr=0.01, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=None, 
    schedule_decay=0.01)


model.compile(
    loss="mse",
    optimizer=adam,
    metrics=["mse"])

train_history = model.fit(
    x=train_x,
    y=train_y,
    epochs=200,
    validation_data=(validation_x, validation_y),
    batch_size=64,
    shuffle=True,
    verbose=1)

Train on 96 samples, validate on 17 samples
Epoch 1/200
96/96 [==============================] - 6s 61ms/step - loss: 0.0976 - mean_squared_error: 0.0976 - val_loss: 0.0809 - val_mean_squared_error: 0.0809
Epoch 2/200
96/96 [==============================] - 0s 2ms/step - loss: 0.0847 - mean_squared_error: 0.0847 - val_loss: 0.0777 - val_mean_squared_error: 0.0777
Epoch 3/200
96/96 [==============================] - 0s 2ms/step - loss: 0.0801 - mean_squared_error: 0.0801 - val_loss: 0.0742 - val_mean_squared_error: 0.0742
Epoch 4/200
96/96 [==============================] - 0s 2ms/step - loss: 0.0791 - mean_squared_error: 0.0791 - val_loss: 0.0755 - val_mean_squared_error: 0.0755
Epoch 5/200
96/96 [==============================] - 0s 2ms/step - loss: 0.0783 - mean_squared_error: 0.0783 - val_loss: 0.0732 - val_mean_squared_error: 0.0732
Epoch 6/200
96/96 [==============================] - 0s 2ms/step - loss: 0.0779 - mean_squared_error: 0.0779 - val_loss: 0.0728 - val_mean_squared_err